In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_percentage_error

from prophet import Prophet
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from statsmodels.tsa.arima.model import ARIMA

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)

# Прогнозирование продаж Walmart: формирование признаков и сравнение моделей

В этом ноутбуке формирую признаки для временного ряда и обучаю несколько моделей:

Модели:
- Seasonal Naive (базовый ориентир)
- ARIMA (один магазин)
- Prophet (один магазин)
- Random Forest (вся сеть)
- Gradient Boosting (вся сеть)

Цель — сравнить точность моделей и определить, какие подходы работают лучше на данных Walmart.


## 1. Основные выводы из анализа данных

Ранее в анализе было выявлено:

- Сильная годовая сезонность.
- Магазины отличаются по уровню продаж.
- Распределение продаж асимметричное, логарифм помогает стабилизировать.
- Праздничные недели влияют на продажи.
- Есть сезонность по месяцам и неделям года.
- Макроэкономические признаки можно использовать в качестве дополнительных фичей.

На основе этих выводов формируются признаки и выбираются модели ниже.


## 2. Загрузка данных и базовая подготовка


In [7]:
df = pd.read_csv('Walmart_Sales.csv')

# Приведение даты
df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")

# Сортировка
df = df.sort_values(["Store", "Date"]).reset_index(drop=True)

# Календарные признаки
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["WeekOfYear"] = df["Date"].dt.isocalendar().week.astype(int)

# Булевый флаг праздника
df["IsHoliday"] = df["Holiday_Flag"] == 1

# Логарифм продаж (для ML-моделей может быть полезен)
df["Weekly_Sales_log"] = np.log1p(df["Weekly_Sales"])

df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,WeekOfYear,IsHoliday,Weekly_Sales_log
0,1,2010-02-05,"1,643,690.90",0,42.31,2.57,211.10,8.11,2010,2,5,False,14.31
1,1,2010-02-12,"1,641,957.44",1,38.51,2.55,211.24,8.11,2010,2,6,True,14.31
2,1,2010-02-19,"1,611,968.17",0,39.93,2.51,211.29,8.11,2010,2,7,False,14.29
3,1,2010-02-26,"1,409,727.59",0,46.63,2.56,211.32,8.11,2010,2,8,False,14.16
4,1,2010-03-05,"1,554,806.68",0,46.50,2.62,211.35,8.11,2010,3,9,False,14.26


## 3. Формирование временных лагов и скользящих средних


In [25]:
def make_lags(df, target="Weekly_Sales", lags=(1, 2, 3, 52)):
    result = df.copy()
    for lag in lags:
        col_name = f"{target}_lag_{lag}"
        result[col_name] = (
            result.groupby("Store")[target]
            .shift(lag)
        )
    return result

def make_rolling(df, target="Weekly_Sales", windows=(4, 12)):
    result = df.copy()
    for w in windows:
        col_name = f"{target}_rolling_{w}"
        result[col_name] = (
            result.groupby("Store")[target]
            .shift(1)
            .rolling(w)
            .mean()
        )
    return result

df = make_lags(df, target="Weekly_Sales", lags=(1, 2, 3, 52))
df = make_rolling(df, target="Weekly_Sales", windows=(4, 12))

df.head()



,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,WeekOfYear,IsHoliday,Weekly_Sales_log,Weekly_Sales_lag_1,Weekly_Sales_lag_2,Weekly_Sales_lag_3,Weekly_Sales_lag_52,Weekly_Sales_rolling_4,Weekly_Sales_rolling_12,Week
0,1,2010-02-05,"1,643,690.90",0,42.31,2.57,211.10,8.11,2010,2,5,False,14.31,NaN,NaN,NaN,NaN,NaN,NaN,5
1,1,2010-02-12,"1,641,957.44",1,38.51,2.55,211.24,8.11,2010,2,6,True,14.31,"1,643,690.90",NaN,NaN,NaN,NaN,NaN,6
2,1,2010-02-19,"1,611,968.17",0,39.93,2.51,211.29,8.11,2010,2,7,False,14.29,"1,641,957.44","1,643,690.90",NaN,NaN,NaN,NaN,7
3,1,2010-02-26,"1,409,727.59",0,46.63,2.56,211.32,8.11,2010,2,8,False,14.16,"1,611,968.17","1,641,957.44","1,643,690.90",NaN,NaN,NaN,8
4,1,2010-03-05,"1,554,806.68",0,46.50,2.62,211.35,8.11,2010,3,9,False,14.26,"1,409,727.59","1,611,968.17","1,641,957.44",NaN,"1,576,836.02",NaN,9


## 4. Разделение на train и test по времени


In [11]:
def train_test_split_by_time(df, test_size=0.1):
    train_parts = []
    test_parts = []
    
    for store_id, part in df.groupby("Store"):
        part = part.sort_values("Date")
        n = len(part)
        split_idx = int(n * (1 - test_size))
        train_parts.append(part.iloc[:split_idx])
        test_parts.append(part.iloc[split_idx:])
    
    train_df = pd.concat(train_parts).reset_index(drop=True)
    test_df = pd.concat(test_parts).reset_index(drop=True)
    return train_df, test_df


train_df, test_df = train_test_split_by_time(df, test_size=0.1)
train_df.shape, test_df.shape


((5760, 19), (675, 19))

## 5. Метрики


In [12]:
def mape(y_true, y_pred):
    return mean_absolute_percentage_error(y_true, y_pred)


def wape(y_true, y_pred):
    num = np.sum(np.abs(y_true - y_pred))
    den = np.sum(np.abs(y_true))
    return num / den


## 6. Модель 1 — Seasonal Naive
Использует значение за ту же неделю год назад.


In [26]:
test_naive = test_df.dropna(subset=["Weekly_Sales_lag_52"]).copy()
test_naive["pred_naive"] = test_naive["Weekly_Sales_lag_52"]

mape_naive = mape(test_naive["Weekly_Sales"], test_naive["pred_naive"])
wape_naive = wape(test_naive["Weekly_Sales"], test_naive["pred_naive"])

mape_naive, wape_naive


(0.05345840401712918, np.float64(0.05067835567361314))

## 7. Модель 2 — ARIMA (1 магазин)


In [29]:
store1 = df[df["Store"] == 1].set_index("Date")
train_s1 = store1.iloc[:-20]
test_s1 = store1.iloc[-20:]

arima_model = ARIMA(train_s1["Weekly_Sales"], order=(2, 1, 2))
arima_res = arima_model.fit()

arima_pred = arima_res.forecast(steps=20)

mape_arima = mape(test_s1["Weekly_Sales"], arima_pred)
wape_arima = wape(test_s1["Weekly_Sales"], arima_pred)

mape_arima, wape_arima


C:\Users\nikita\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-FRI will be used.
  self._init_dates(dates, freq)
C:\Users\nikita\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-FRI will be used.
  self._init_dates(dates, freq)
C:\Users\nikita\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-FRI will be used.
  self._init_dates(dates, freq)


(0.06004801414387073, np.float64(0.05878328392949088))

## 8. Модель 3 — Prophet (1 магазин)


In [30]:
prophet_df = store1.reset_index()[["Date", "Weekly_Sales"]].rename(
    columns={"Date": "ds", "Weekly_Sales": "y"}
)

train_p = prophet_df.iloc[:-20]
test_p = prophet_df.iloc[-20:]

m_prophet = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True
)

m_prophet.fit(train_p)
future = m_prophet.make_future_dataframe(periods=20, freq="W")
forecast = m_prophet.predict(future)

pred_prophet = forecast.iloc[-20:]["yhat"].values

mape_prophet = mape(test_p["y"], pred_prophet)
wape_prophet = wape(test_p["y"], pred_prophet)

mape_prophet, wape_prophet


22:32:26 - cmdstanpy - INFO - Chain [1] start processing
22:32:26 - cmdstanpy - INFO - Chain [1] done processing


(0.10384443222136903, np.float64(0.10191063370266332))

## 9. Модель 4 — Random Forest (все магазины)


In [32]:
df["Week"] = df["Date"].dt.isocalendar().week.astype(int)

rf_features = [
    "Weekly_Sales_lag_1",
    "Weekly_Sales_lag_2",
    "Weekly_Sales_lag_3",
    "Month",
    "Week",
    "CPI",
    "Unemployment",
]

train_rf, test_rf = train_test_split_by_time(
    df.dropna(subset=rf_features)
)

X_train = train_rf[rf_features]
y_train = train_rf["Weekly_Sales"]
X_test = test_rf[rf_features]
y_test = test_rf["Weekly_Sales"]

rf_model = RandomForestRegressor(n_estimators=300, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

mape_rf = mape(y_test, rf_pred)
wape_rf = wape(y_test, rf_pred)

mape_rf, wape_rf


(0.045323073543083355, np.float64(0.04474755911929917))

## 10. Модель 5 — Gradient Boosting (все магазины)


In [35]:
gb_model = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3
)

gb_model.fit(X_train, y_train)
gb_pred = gb_model.predict(X_test)

mape_gb = mape(y_test, gb_pred)
wape_gb = wape(y_test, gb_pred)

mape_gb, wape_gb


(0.04192333524972996, np.float64(0.040489907548538887))

## 11. Сравнение моделей


In [36]:
results = pd.DataFrame([
    ["Seasonal Naive", mape_naive, wape_naive],
    ["ARIMA (1 магазин)", mape_arima, wape_arima],
    ["Prophet (1 магазин)", mape_prophet, wape_prophet],
    ["Random Forest", mape_rf, wape_rf],
    ["Gradient Boosting", mape_gb, wape_gb],
], columns=["Model", "MAPE", "WAPE"])

results


,Model,MAPE,WAPE
0,Seasonal Naive,0.05,0.05
1,ARIMA (1 магазин),0.06,0.06
2,Prophet (1 магазин),0.10,0.10
3,Random Forest,0.05,0.04
4,Gradient Boosting,0.04,0.04


# Итоги

В этом ноутбуке:

- сформированы временные и календарные признаки;
- выполнено разбиение по времени для каждой модели;
- обучены модели разных типов: статистические, ML и бустинг;
- получены метрики MAPE и WAPE.

Gradient Boosting показывает наименьшую ошибку среди протестированных моделей.  
Random Forest также даёт стабильный результат на уровне всей сети.

Этот набор моделей демонстрирует полный цикл: от простого baseline до сильной ML-модели.

Лучше всего работает глобальная модель, обученная на всей сети. 

- Модели, построенные на одном магазине, показывают стабильный результат, но хуже, чем глобальные модели по всей сети.
- Глобальная модель переносит полученный опыт между магазинами.
